In [1]:
include("../src/MTJWAS.jl")
using MTJWAS

## <font color="blue"> Data</font>

In [2]:
using DataFrames

In [3]:
DIR="data/"
phenofile=DIR*"phenotype.txt"
genofile =DIR*"genotype.txt"
pedfile  =DIR*"pedigree.txt";

### phenotypes

In [4]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [5]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [6]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


### Genetic covariance matrix and residual covariance matrix

In [7]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

## <font color="blue">1. mulitiple trait with fixed effects</font> 

In [8]:
using MTJWAS

In [9]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


#### set up model equations

In [10]:
model_equations = "BW = intercept + age + sex;
                  CW = intercept + age + sex";

In [12]:
model1 = MTJWAS.build_model(model_equations,R);

#### set variables as covariate (all variables are set as factors by default)

In [13]:
MTJWAS.set_covariate(model1,"age")

#### run

In [14]:
out1=runMCMC(model1,data,chain_length=50000,printout_frequency=10000);

at sample: 10000
Residual covariance matrix: 
[140.154850433725 33.53738407776266
 33.53738407776266 8.478479416354546]

at sample: 20000
Residual covariance matrix: 
[146.39527147436098 35.155128514927284
 35.155128514927284 8.886245535327399]

at sample: 30000
Residual covariance matrix: 
[146.59006151035737 35.257743533327066
 35.257743533327066 8.928910545337384]

at sample: 40000
Residual covariance matrix: 
[144.6446141420062 34.729994318875356
 34.729994318875356 8.788948212738667]

at sample: 50000
Residual covariance matrix: 
[145.4376704484496 34.92639206058722
 34.92639206058722 8.839095337975094]



## <font color="blue">2. multiple trait with direct genetic effects</font> 

#### set up model equations

In [15]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [16]:
model2    = MTJWAS.build_model(model_equations,R);

In [18]:
MTJWAS.set_covariate(model1,"age")

#### set variables as random variables

* random variables whose covarince matrices are numerator relationship matrix 

In [19]:
ped = MTJWAS.get_pedigree(pedfile);

In [20]:
MTJWAS.set_random(model2,"Animal", ped,G)

#### run

In [21]:
out2 = MTJWAS.runMCMC(model2,data,chain_length=50000,printout_frequency=10000);

at sample: 10000
Residual covariance matrix: 
[9.820165943766979 2.0046236011310343
 2.0046236011310343 1.0144411655137777]

at sample: 20000
Residual covariance matrix: 
[9.65898383277881 1.96129223479615
 1.96129223479615 0.9982560556271673]

at sample: 30000
Residual covariance matrix: 
[9.704919994056782 1.9660159105243893
 1.9660159105243893 0.9893309657009017]

at sample: 40000
Residual covariance matrix: 
[9.631503460472059 1.9548198779333683
 1.9548198779333683 0.9930878018448202]

at sample: 50000
Residual covariance matrix: 
[9.708558625560551 1.9522645726047894
 1.9522645726047894 0.9882216326407085]



## <font color="blue">3. mulitiple traits with marker effects</font> 

#### set up model equations

In [22]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [23]:
model3    = MTJWAS.build_model(model_equations,R);

In [25]:
MTJWAS.set_covariate(model3,"age")

In [26]:
MTJWAS.add_markers(model3,genofile,G,separator=',',header=true);

The delimiters in file data/genotype.txt is ,  .


In [27]:
out3=MTJWAS.runMCMC(model3,data,chain_length=50000,printout_frequency=10000);

at sample: 10000
Residual covariance matrix: 
[43.37990025135148 10.43779421325325
 10.43779421325325 3.2443567269783204]
Marker effects covariance matrix: 
[34.712159075131765 6.63560426385648
 6.63560426385648 2.179277074198228]

at sample: 20000
Residual covariance matrix: 
[44.79762004520499 10.745526578100506
 10.745526578100506 3.2901324663387843]
Marker effects covariance matrix: 
[37.2835369158311 7.3599874279394095
 7.3599874279394095 2.3520826784581534]

at sample: 30000
Residual covariance matrix: 
[41.540832692269205 9.878648952258995
 9.878648952258995 3.063809968040247]
Marker effects covariance matrix: 
[36.92118133337858 7.364193554874619
 7.364193554874619 2.373864976268515]

at sample: 40000
Residual covariance matrix: 
[52.710015727285395 13.339957838119721
 13.339957838119721 4.166635036680465]
Marker effects covariance matrix: 
[36.68224550584762 7.3372716378947755
 7.3372716378947755 2.3957266827966652]

at sample: 50000
Residual covariance matrix: 
[51.5506040952

## <font color="blue">4. mulitiple traits with marker effects and polygenic effects</font> 

#### set up model equations

In [28]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [29]:
model4    = MTJWAS.build_model(model_equations,R);

In [30]:
MTJWAS.set_covariate(model4,"age")

In [31]:
GA = G*0.1
MTJWAS.set_random(model4,"Animal",ped,GA)

In [32]:
GM = G*0.9
MTJWAS.add_markers(model4,genofile,GM,separator=',',header=true);

The delimiters in file data/genotype.txt is ,  .


#### run

In [33]:
out4=MTJWAS.runMCMC(model4,data,chain_length=50000,printout_frequency=10000);

at sample: 10000
Residual covariance matrix: 
[35.48572233277769 8.282656696068331
 8.282656696068331 2.6299397400892617]
Marker effects covariance matrix: 
[34.52682140161455 6.847668774876981
 6.847668774876981 2.2063706088382373]

at sample: 20000
Residual covariance matrix: 
[49.20041389539753 11.601222504923568
 11.601222504923568 3.4573659627427866]
Marker effects covariance matrix: 
[30.043909434730566 5.794357624368241
 5.794357624368241 1.9422225670329665]

at sample: 30000
Residual covariance matrix: 
[43.83645040824476 10.390115491195754
 10.390115491195754 3.1675496004900987]
Marker effects covariance matrix: 
[34.874146370642976 6.913474711866261
 6.913474711866261 2.1978146019015483]

at sample: 40000
Residual covariance matrix: 
[46.601025980142 11.126368299744152
 11.126368299744152 3.3643141880743825]
Marker effects covariance matrix: 
[33.124278656110185 6.555156284874134
 6.555156284874134 2.1255066048960645]

at sample: 50000
Residual covariance matrix: 
[48.7456725

## <font color="blue">5. check results</font> 

In [34]:
keys(out4)

Base.KeyIterator for a Dict{Any,Any} with 5 entries. Keys:
  "posterior mean of marker effects covariance matrix"
  "posterior mean of residual covaraince matrix"
  "posterior mean of polygenic effects covariance matrix"
  "posterior mean of marker effects"
  "posterior mean of location parameters"

In [35]:
out4["posterior mean of polygenic effects covariance matrix"]

2x2 Array{Float64,2}:
 2.12913   0.124127
 0.124127  0.199814

In [36]:
out4["posterior mean of location parameters"]

18x2 Array{Any,2}:
 "1:intercept: intercept"  -164.375      
 "1:age: age"                -6.20652    
 "1:sex: M"                 320.498      
 "1:sex: F"                 246.396      
 "1:Animal: S1"              -0.2143     
 "1:Animal: D1"               0.220048   
 "1:Animal: O1"               0.0952816  
 "1:Animal: O3"              -0.131879   
 "1:Animal: O2"               0.000732052
 "2:intercept: intercept"    45.7918     
 "2:age: age"                 0.308842   
 "2:sex: M"                 -36.6861     
 "2:sex: F"                 -37.8728     
 "2:Animal: S1"              -0.0554943  
 "2:Animal: D1"               0.0473173  
 "2:Animal: O1"               0.0112568  
 "2:Animal: O3"              -0.0327187  
 "2:Animal: O2"              -0.00502265 

In [37]:
out4["posterior mean of marker effects"][1]

5x2 Array{Any,2}:
 "x1"   6.77731
 "x2"   4.19905
 "x3"  -1.68684
 "x4"   3.47748
 "x5"  -2.09433

In [38]:
out4["posterior mean of marker effects"][2]

5x2 Array{Any,2}:
 "x1"   1.57689 
 "x2"   0.92599 
 "x3"  -0.416727
 "x4"   0.800486
 "x5"  -0.490916

## <font color="blue">Bonus: maternal effects</font> 

In [39]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [40]:
d2 = [data[3:4,:]  DataFrame(Dam = ["D1","D1"])]

,Animal,BW,CW,age,sex,Dam
1,O1,150.0,13.0,3,M,D1
2,O3,40.0,5.0,4,F,D1


In [41]:
model_equations = "BW = intercept + age + sex + Animal+ Dam;
                   CW = intercept + age + sex + Animal";

In [42]:
model5 = MTJWAS.build_model(model_equations,R);

In [43]:
MTJWAS.set_covariate(model1,"age")

In [44]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
MTJWAS.set_random(model5,"Animal Dam", ped,G0)

In [45]:
out5=MTJWAS.runMCMC(model5,d2,chain_length=50000,printout_frequency=10000);

at sample: 10000
Residual covariance matrix: 
[9.873410187700825 1.9979197257512247
 1.9979197257512247 0.9955478501770609]

at sample: 20000
Residual covariance matrix: 
[9.82039106006733 1.9643322260636862
 1.9643322260636862 0.9820294682288431]

at sample: 30000
Residual covariance matrix: 
[9.850931588079236 1.9558271641426421
 1.9558271641426421 0.9849899325582364]

at sample: 40000
Residual covariance matrix: 
[9.923554264162794 1.9774158506781074
 1.9774158506781074 0.9905738801295279]

at sample: 50000
Residual covariance matrix: 
[9.883671914052716 1.9684451715352882
 1.9684451715352882 0.9867708894909825]



In [46]:
#mme.pedTrmVec